## Sales Report Top Departments Query

In [7]:
import pandas as pd
import psycopg2

def query_sales_report_data(connection_string):
    """
    Queries employee payment data and returns the results as a Pandas DataFrame.

    Args:
        connection_string (str): PostgreSQL connection string.

    Returns:
        pd.DataFrame: DataFrame containing the query results.
    """
    query = """
    SELECT
        t.store_id,
        DATE_TRUNC('month', t.transaction_date)::date AS sale_month,
        d.dept_name,
        SUM(s.quantity) AS total_quantity_sold,
        COUNT(DISTINCT t.transaction_id) AS total_transactions
    FROM
        sales s
    INNER JOIN transactions t
        ON s.transaction_id = t.transaction_id
    INNER JOIN products p
        ON s.product_id = p.product_id
    INNER JOIN department d
        ON p.dept_id = d.dept_id
    WHERE
        t.transaction_type = 0 -- Include only sales (exclude returns)
    GROUP BY
        t.store_id, sale_month, d.dept_name
    ORDER BY
        t.store_id, sale_month, d.dept_name;

    """
    
    # Connect to the database
    with psycopg2.connect(connection_string) as conn:
        # Use Pandas to execute the query and return the results as a DataFrame
        df = pd.read_sql_query(query, conn)
    
    return df




In [ ]:
# Example Usage
connection_string = "dbname='abc_foodmart' user='postgres' password='jade5808' host='localhost' port='5432'"
store_sales = query_sales_report_data(connection_string)

print(store_sales)

    store_id  sale_month           dept_name  total_quantity_sold  \
0          1  2024-10-01              Bakery                   65   
1          1  2024-10-01           Beverages                   63   
2          1  2024-10-01               Dairy                   35   
3          1  2024-10-01        Frozen Foods                   84   
4          1  2024-10-01     Health & Beauty                   99   
..       ...         ...                 ...                  ...   
85         3  2024-12-01  Household Supplies                   18   
86         3  2024-12-01      Meat & Seafood                   17   
87         3  2024-12-01              Pantry                    5   
88         3  2024-12-01             Produce                    9   
89         3  2024-12-01              Snacks                   16   

    total_transactions  
0                   17  
1                   18  
2                   13  
3                   28  
4                   28  
..                 ..

/var/folders/4c/4dspdyxs01gbnk6gwhydwpnr0000gn/T/ipykernel_19609/288021475.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


## Sales Report Top Products from Each Store

In [16]:
import pandas as pd
import psycopg2

def query_top_worst_products(connection_string):
    """
    Queries employee payment data and returns the results as a Pandas DataFrame.

    Args:
        connection_string (str): PostgreSQL connection string.

    Returns:
        pd.DataFrame: DataFrame containing the query results.
    """
    query = """
WITH product_sales AS (
    SELECT
        t.store_id,
        s.store_name,
        p.product_id,
        p.product_name,
        SUM(sales.quantity) AS total_quantity_sold
    FROM
        sales
    INNER JOIN transactions t
        ON sales.transaction_id = t.transaction_id
    INNER JOIN store s
        ON t.store_id = s.store_id
    INNER JOIN products p
        ON sales.product_id = p.product_id
    WHERE
        t.transaction_type = 0 -- Include only sales (exclude returns)
    GROUP BY
        t.store_id, s.store_name, p.product_id, p.product_name
),
ranked_sales AS (
    SELECT
        ps.store_id,
        ps.store_name,
        ps.product_id,
        ps.product_name,
        ps.total_quantity_sold,
        RANK() OVER (PARTITION BY ps.store_id ORDER BY ps.total_quantity_sold DESC) AS rank_best,
        RANK() OVER (PARTITION BY ps.store_id ORDER BY ps.total_quantity_sold ASC) AS rank_least
    FROM
        product_sales ps
)
SELECT
    store_id,
    store_name,
    product_id,
    product_name,
    total_quantity_sold,
    CASE
        WHEN rank_best <= 5 THEN 'Top-Selling'
        WHEN rank_least <= 5 THEN 'Least-Selling'
    END AS product_category
FROM
    ranked_sales
WHERE
    rank_best <= 5 OR rank_least <= 5
ORDER BY
    store_id, product_category, total_quantity_sold DESC;


    """
    
    # Connect to the database
    with psycopg2.connect(connection_string) as conn:
        # Use Pandas to execute the query and return the results as a DataFrame
        df = pd.read_sql_query(query, conn)
    
    return df




In [17]:
# Example Usage
connection_string = "dbname='abc_foodmart' user='postgres' password='jade5808' host='localhost' port='5432'"
products = query_top_worst_products(connection_string)

# Display the DataFrame
print(products)

    store_id              store_name  product_id  \
0          1  ABC Food Mart Queens 1          90   
1          1  ABC Food Mart Queens 1          41   
2          1  ABC Food Mart Queens 1          35   
3          1  ABC Food Mart Queens 1           1   
4          1  ABC Food Mart Queens 1          97   
5          1  ABC Food Mart Queens 1          11   
6          1  ABC Food Mart Queens 1          51   
7          1  ABC Food Mart Queens 1          77   
8          1  ABC Food Mart Queens 1          27   
9          1  ABC Food Mart Queens 1          64   
10         1  ABC Food Mart Queens 1          98   
11         1  ABC Food Mart Queens 1          31   
12         1  ABC Food Mart Queens 1           8   
13         1  ABC Food Mart Queens 1          86   
14         2  ABC Food Mart Queens 2          68   
15         2  ABC Food Mart Queens 2           4   
16         2  ABC Food Mart Queens 2           1   
17         2  ABC Food Mart Queens 2          80   
18         2

/var/folders/4c/4dspdyxs01gbnk6gwhydwpnr0000gn/T/ipykernel_19609/2951614280.py:70: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


## Vendor Delivery Tracking

In [9]:
import pandas as pd
import psycopg2

def query_delivery_data(connection_string):
    """
    Queries employee payment data and returns the results as a Pandas DataFrame.

    Args:
        connection_string (str): PostgreSQL connection string.

    Returns:
        pd.DataFrame: DataFrame containing the query results.
    """
    query = """
    SELECT
        v.vendor_id,
        v.vendor_name,
        p.product_id,
        p.product_name,
        s.store_id,
        s.store_name,
        d.delivery_date,
        d.quantity
    FROM
        deliveries d
    INNER JOIN products p
        ON d.product_id = p.product_id
    INNER JOIN vendors v
        ON d.vendor_id = v.vendor_id
    INNER JOIN store s
        ON d.store_id = s.store_id
    WHERE 
        v.vendor_id = 3
    AND d.delivery_date BETWEEN '2024-11-01' AND '2024-12-01'
    ORDER BY
        v.vendor_id, d.delivery_date DESC;

    """
    
    # Connect to the database
    with psycopg2.connect(connection_string) as conn:
        # Use Pandas to execute the query and return the results as a DataFrame
        df = pd.read_sql_query(query, conn)
    
    return df




In [11]:
# Example Usage
connection_string = "dbname='abc_foodmart' user='postgres' password='jade5808' host='localhost' port='5432'"
delivery = query_delivery_data(connection_string)

# Display the DataFrame
print(delivery)

    vendor_id           vendor_name  product_id                 product_name  \
0           3  Jones, Johnston and           90         GoodFood Body Lotion   
1           3  Jones, Johnston and            8       PureTaste Canned Beans   
2           3  Jones, Johnston and           27    GreenValley Bottled Water   
3           3  Jones, Johnston and           71     GreenValley Potato Chips   
4           3  Jones, Johnston and            2          HarvestBest Shampoo   
5           3  Jones, Johnston and           24  NatureDelight Salmon Fillet   
6           3  Jones, Johnston and           65       PureTaste Greek Yogurt   
7           3  Jones, Johnston and            7     OrganicChoice Whole Milk   
8           3  Jones, Johnston and           85            FreshFarm Popcorn   
9           3  Jones, Johnston and           77        GoodFood Frozen Fries   
10          3  Jones, Johnston and           57           GoodFood Dish Soap   
11          3  Jones, Johnston and      

/var/folders/4c/4dspdyxs01gbnk6gwhydwpnr0000gn/T/ipykernel_19609/3583448531.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


## Employee Pay Periods Query

In [3]:
import pandas as pd
import psycopg2

def query_employee_payment_data(connection_string):
    """
    Queries employee payment data and returns the results as a Pandas DataFrame.

    Args:
        connection_string (str): PostgreSQL connection string.

    Returns:
        pd.DataFrame: DataFrame containing the query results.
    """
    query = """
    SELECT
        ett.employee_id,
        p.pay_period_start,
        p.pay_period_end,
        SUM(EXTRACT(EPOCH FROM (ett.end_time - ett.start_time)) / 3600) AS total_hours_worked,
        p.hour_wage,
        SUM(EXTRACT(EPOCH FROM (ett.end_time - ett.start_time)) / 3600) * p.hour_wage AS total_amount_paid
    FROM
        employee_time_tracking ett
    INNER JOIN payroll p
        ON ett.employee_id = p.employee_id
        AND ett.start_time::date BETWEEN p.pay_period_start AND p.pay_period_end
    WHERE
        ett.start_time::date >= '2023-11-01' AND ett.start_time::date <= CURRENT_DATE
    GROUP BY
        ett.employee_id, p.pay_period_start, p.pay_period_end, p.hour_wage
    ORDER BY
        ett.employee_id, p.pay_period_start;
    """
    
    # Connect to the database
    with psycopg2.connect(connection_string) as conn:
        # Use Pandas to execute the query and return the results as a DataFrame
        df = pd.read_sql_query(query, conn)
    
    return df




In [4]:
# Example Usage
connection_string = "dbname='abc_foodmart' user='postgres' password='jade5808' host='localhost' port='5432'"
employee_payment_df = query_employee_payment_data(connection_string)

# Display the DataFrame
print(employee_payment_df)

     employee_id pay_period_start pay_period_end  total_hours_worked  \
0              1       2024-11-11     2024-11-24                82.0   
1              1       2024-11-25     2024-12-08                78.0   
2              2       2024-11-11     2024-11-24                83.0   
3              2       2024-11-25     2024-12-08                78.0   
4              3       2024-11-11     2024-11-24                84.0   
..           ...              ...            ...                 ...   
175           88       2024-11-25     2024-12-08                79.0   
176           89       2024-11-11     2024-11-24                97.0   
177           89       2024-11-25     2024-12-08                82.0   
178           90       2024-11-11     2024-11-24                85.0   
179           90       2024-11-25     2024-12-08                73.0   

     hour_wage  total_amount_paid  
0        29.16            2391.12  
1        29.16            2274.48  
2        23.00            1

/var/folders/4c/4dspdyxs01gbnk6gwhydwpnr0000gn/T/ipykernel_19609/3107706982.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


## Inventory Query

In [12]:
import pandas as pd
import psycopg2

def query_inventory(connection_string):
    """
    Queries employee payment data and returns the results as a Pandas DataFrame.

    Args:
        connection_string (str): PostgreSQL connection string.

    Returns:
        pd.DataFrame: DataFrame containing the query results.
    """
    query = """
    SELECT
        i.store_id,
        s.store_name,
        i.product_id,
        p.product_name,
        i.quantity AS inventory_level,
        i.last_updated
    FROM
        inventory i
    INNER JOIN store s
        ON i.store_id = s.store_id
    INNER JOIN products p
        ON i.product_id = p.product_id
    ORDER BY
        i.store_id, i.product_id;

    """
    
    # Connect to the database
    with psycopg2.connect(connection_string) as conn:
        # Use Pandas to execute the query and return the results as a DataFrame
        df = pd.read_sql_query(query, conn)
    
    return df

In [15]:
# Example Usage
connection_string = "dbname='abc_foodmart' user='postgres' password='jade5808' host='localhost' port='5432'"
inventory = query_inventory(connection_string)

inventory.tail(25)

/var/folders/4c/4dspdyxs01gbnk6gwhydwpnr0000gn/T/ipykernel_19609/972199641.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,store_id,store_name,product_id,product_name,inventory_level,last_updated
275,3,ABC Food Mart Brooklyn,76,GreenValley Croissant,1250,2024-12-08 18:16:53.143942
276,3,ABC Food Mart Brooklyn,77,GoodFood Frozen Fries,1338,2024-12-08 18:16:53.143942
277,3,ABC Food Mart Brooklyn,78,OrganicChoice Apples,1424,2024-12-08 19:54:13.253752
278,3,ABC Food Mart Brooklyn,79,GoodFood Trash Bags,2966,2024-12-08 18:16:53.143942
279,3,ABC Food Mart Brooklyn,80,NatureDelight Paper Towels,2125,2024-12-08 19:54:13.253752
280,3,ABC Food Mart Brooklyn,81,HarvestBest Toothpaste,643,2024-12-08 18:16:53.143942
281,3,ABC Food Mart Brooklyn,82,HarvestBest Baguette,1729,2024-12-08 19:54:13.253752
282,3,ABC Food Mart Brooklyn,83,NatureDelight Pretzels,3706,2024-12-08 18:16:53.143942
283,3,ABC Food Mart Brooklyn,84,GoodFood Popcorn,2247,2024-12-08 18:16:53.143942
284,3,ABC Food Mart Brooklyn,85,FreshFarm Popcorn,816,2024-12-08 18:16:53.143942
